### LexRank algoritam

`LexRank` algoritam je algoritam za ekstraktivnu sumarizaciju teksta koji koristi ideju opisanog `PageRank` algoritma. Zadatak esktraktivne sumarizacije je da za zadati tekst generiše kraću verziju izdvajajući samo rečenice koje su najinformativnije.  <img src='assets/summarization.jpg'>

Čvorovi grafa će sada biti rečenice $s_i$, ivice će povezivati rečenice koje dele informacioni sadržaj, a njihove težine $w_{ij}$ će odgovarati količini preklapanja informacionih sadržaja.  <img src='assets/lexrank_color.png' style='width:400px'>

#### Obrada teksta

Da bismo pripremili tekst za rad, iskoristićemo `nltk` (skraćenica od *natural language toolkit*) paket koji se koristi za rad sa tekstom, a koji nudi podršku za zadatke tokenizacije, tagiranja, normalizacije i slično. Alat se može instalirati komandom `conda install -c anaconda nltk` u skladu sa smernicama sa [zvaničnog sajta](https://anaconda.org/anaconda/nltk). 

Punkt je ime podalata koji se koristi za razbija tekst na rečenice i tokene. 

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/andjelka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

U radu ćemo koristiti članke o tenisu dostupne u `tennis_articles.csv` datoteci. 

In [2]:
import pandas as pd

In [3]:
data =  pd.read_csv('data/tennis_articles.csv')

Svaki članak ima svoj identifikator, tekst i izvor.

In [4]:
 data.columns

Index(['article_id', 'article_text', 'source'], dtype='object')

Npr. jedan od članaka u kolekciji je: 

In [5]:
data['article_text'][1]

"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with

i njega ćemo iskoristiti za dalje testiranje.

In [6]:
article = data['article_text'][1]

Ukupan broj članaka kolekcije je: 

In [7]:
data.shape[0]

8

Koraci koje ćemo preduzeti za svaki članak prikazani su na slici. <img src='assets/summarization_pipeline.png' style='width:500px'> Mi ćemo raditi nad pojedinačnim člancima pa je potrebno preskočiti korak *combine*.

Pojedinačne tekstove ćemo razbiti na rečenice korišćenjem `nltk` paketa i njegove funkcije `sent_tokenize`. 

In [8]:
sentences = nltk.sent_tokenize(article)

In [9]:
sentences

['BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday.',
 'Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday.',
 'Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1.',
 "He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand.",
 'He clinched on his fourth chance when Medvedev netted from the baseline.',
 'Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal.',
 'The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.',
 'Copil has two after also beating No.',
 '6 Marin Cilic in the second round.',
 'Copil fired 26 aces past Zverev and never dropped se

Dalje ćemo na nivou rečenica izdvojiti pojedinačne reči. Prilikom izdvajanja reči eliminisaćemo interpunkciju i takozvane stop reči koje se jako često pojavljuju i koje ne doprinose svojim značenjem (npr. reči poput članova ili predloga).

Za izdvajanje reči koristićemo funkciju `word_tokenize`. Ova funkcija očekuje segment teksta, a kao rezultat vraća listu reči.

In [10]:
tokens = nltk.word_tokenize("Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago.")

In [11]:
tokens

['Federer',
 'had',
 'an',
 'easier',
 'time',
 'than',
 'in',
 'his',
 'only',
 'previous',
 'match',
 'against',
 'Medvedev',
 ',',
 'a',
 'three-setter',
 'at',
 'Shanghai',
 'two',
 'weeks',
 'ago',
 '.']

Stop reči možemo pročitati iz paketa `stopwords` za svaki od jezika koji podržava biblioteka `nltk`.

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andjelka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
english_stopwords = stopwords.words('english')

Na primer, neke od reči koje se smatraju stop rečima su:

In [14]:
english_stopwords[0:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

Interpunkcijske karaktere ćemo očitati iz `string` paketa.

In [15]:
import string

In [16]:
punctuation = string.punctuation

In [17]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Na gore opisane načine, sada ćemo pripremiti izdvojene rečenice.

In [18]:
clean_sentences = []

for sentence in sentences:

    # 1) izdvajamo pojedinačne reci u recenici
    tokens = nltk.word_tokenize(sentence)

    # 2) normalizujemo reci - u nasem slucaju ce to biti svodjenje na zapis malim slovima
    normalized_tokens = [token.lower() for token in tokens]

    # 3) zadrzavamo samo one reci koje nisu u skupu stop reci i koji ne predstavljaju interpunkciju
    clean_tokens = [
        token for token in normalized_tokens
        if token not in english_stopwords and token not in punctuation
    ]

    # 4) spajamo reci u recenicu
    clean_sentence = " ".join(clean_tokens)

    # 5) recenicu dodajemo skupu pripremljenih recenica
    if len(clean_sentence) != 0:
        clean_sentences.append(clean_sentence)

In [19]:
clean_sentences

['basel switzerland ap roger federer advanced 14th swiss indoors final career beating seventh-seeded daniil medvedev 6-1 6-4 saturday',
 'seeking ninth title hometown event 99th overall federer play 93th-ranked marius copil sunday',
 'federer dominated 20th-ranked medvedev first match-point chance break serve 5-1',
 "dropped serve love let another match point slip medvedev 's next service game netting backhand",
 'clinched fourth chance medvedev netted baseline',
 'copil upset expectations federer final alexander zverev 6-3 6-7 6 6-4 win fifth-ranked german earlier semifinal',
 'romanian aims first title arriving basel without career win top-10 opponent',
 'copil two also beating',
 '6 marin cilic second round',
 'copil fired 26 aces past zverev never dropped serve clinching 2 1/2 hours forehand volley winner break zverev second time semifinal',
 "came two rounds qualifying last weekend reach basel main draw including beating zverev 's older brother mischa",
 'federer easier time previ

Da bismo dobili vektorsku reprezentaciju teksta, iskoristićemo GloVe pritrenirane distribuirane reprezentacije reči. Alternativa su mogli biti i n-gramski profili rečenica ili njihove Tf-Idf reprezentacije, ali se u praksi dobijaju bolji rezultati ukoliko se iskoriste ovakve reprezentacije. Ceo paket sa reprezentacijama reči različitih dužina (50, 100, 200 i 300) ukupne veličine 822MB se može preuzeti sa [zvanične adrese](https://nlp.stanford.edu/projects/glove/). Mi ćemo u radu koristiti reprezentacije dužine 100 koje se nalaze u datoteci `glove.6B.100d.txt` (347.1MB) koja se može preuzeti pojedinačno npr. sa [ove](https://www.kaggle.com/terenceliu4444/glove6b100dtxt) adrese. 

Prvo ćemo pročitati iz preuzete datoteke sve podržane reči i njihove vektorske reprezentacije. U pojedinačnim redovima datoteke se prvo nalazi reči, a potom 100 realnih vrednosti koje predstavljaju njenu vektorsku reprezentaciju. 

In [20]:
import numpy as np

In [21]:
embedding_size = 100

In [22]:
word_embeddings = {}
with open('data/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

Broj reči koji je podržan ovim reprezentacijama je:

In [23]:
len(word_embeddings)

400000

Na primer, vektorska reprezentacija reči *tennis* se može dobiti pomoću funkcije `get`:

In [24]:
word_embeddings.get('tennis')

array([ 0.21508  ,  0.61981  ,  0.84039  ,  0.71394  , -0.29904  ,
        0.56481  ,  0.18241  ,  0.76767  , -0.75897  , -0.056711 ,
        0.43726  , -0.39217  , -0.14874  ,  0.19475  , -0.69581  ,
        0.58388  ,  0.20625  ,  0.36635  , -0.36793  ,  0.68765  ,
       -0.5191   ,  0.92246  ,  0.6831   ,  0.92039  ,  0.31221  ,
        0.10465  ,  0.253    , -1.9131   ,  0.67281  ,  0.38894  ,
       -0.88199  ,  0.22536  ,  0.027648 , -0.55574  ,  0.43641  ,
       -0.18579  , -1.3131   ,  1.1555   , -1.2937   , -0.46866  ,
        0.16292  , -0.28636  ,  0.25793  , -1.3538   ,  0.28808  ,
       -0.040711 ,  0.027864 ,  0.21767  ,  0.8588   , -0.98463  ,
       -0.73366  , -0.6457   ,  0.61292  ,  0.23316  ,  0.42164  ,
       -1.8037   , -0.0055624,  1.0998   ,  0.9493   ,  1.0987   ,
       -0.52362  ,  0.49657  , -0.23824  ,  0.52824  , -0.54642  ,
       -0.48527  , -0.42944  ,  0.25497  , -0.16199  ,  0.018633 ,
        0.12416  , -0.13436  , -0.082436 , -0.042168 , -0.1311

Ukoliko se reč ne nalazi u podržanoj listi reči, kao drugi argument funkcije `get` se može navesti podrazumevana reprezentacija. U našem slučaju to će biti vektor nula dužine 100. Za reči koje se ne nalaze u podržanoj listi reči kažemo da su reči izvan vokabulara (engl. out of vocabulary words).

In [25]:
oov_word = np.zeros((embedding_size,))

Vektorsku reprezentaciju rečenice dobićemo uprosečavanjem vektorskih reprezentacija njenih reči. Ovako dobijene vektore ćemo sačuvati u matrici čija je dimenzija `broj rečenica x dužina ugnježdene reprezentacije`. Reprezentacije pojedinačnih rečenica će biti smeštene kao vrste matrice.

In [26]:
sentence_vectors = np.zeros((len(clean_sentences), embedding_size))
for i, sentence in enumerate(clean_sentences):
    v = sum(
        [word_embeddings.get(word, oov_word)
         for word in sentence.split()]) / (len(sentence.split()) + 0.001)
    
    sentence_vectors[i, :] = v

In [27]:
sentence_vectors.shape

(12, 100)

#### Kreiranje grafa

Dalje je potrebno pripremiti matricu grafa čiji elementi predstavljaju meru sličnosti između rečenica. Za računanje sličnosti između rečenica koristićemo kosinusnu sličnost. 

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity_matrix = cosine_similarity(sentence_vectors)

Za rangiranje rečenica u sažetku iskoristićemo bibliotečku `pagerank` funkciju. 

In [30]:
import networkx as nx

In [31]:
graph = nx.from_numpy_array(similarity_matrix, create_using=nx.Graph)
scores = nx.pagerank(graph, max_iter=1000)

Dobijeni skorovi rečenica su: 

In [32]:
scores

{0: 0.08240804406547644,
 1: 0.08472309859484514,
 2: 0.08637699645126401,
 3: 0.0837634012841951,
 4: 0.0812589299878768,
 5: 0.08113820392276073,
 6: 0.08414707168824694,
 7: 0.0808923391734895,
 8: 0.07893805099495363,
 9: 0.08602334463420958,
 10: 0.08503104341624794,
 11: 0.08529947578643422}

Ostalo je još sortirati rečenice spram dobijenog porekta i izdvojiti nekoliko njih, na primer 2. 

In [33]:
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

In [34]:
summary_length = 2
for i in range(summary_length):
    print(ranked_sentences[i][1])

Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1.
Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with a forehand volley winner to break Zverev for the second time in the semifinal.


O originalnoj verziji LexRank-a možete pročitati više na [ovoj adresi](https://arxiv.org/pdf/1109.2128.pdf), a sličnu implementaciju sa imenom TextRank možete koristiti u okviru [Gensim biblioteke](https://radimrehurek.com/gensim/summarization/summariser.html). Nju je potrebno instalirati komandom `conda install -c anaconda gensim` u skladu sa [zvaničnim smernicama](https://anaconda.org/anaconda/gensim). Ovde je koristimo sa idejom da uporedimo rezultate jer ova biblioteka koristi Tf-IDf reprezentaciju teksta. 

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
print(summarize(article))

<div class="alert alert-info">
Zadatak za vežbu: 
<br>
Uporediti ponašanje sumarizatora ukoliko se umesto distribuiranih reprezentacija iskoriste Tf-Idf reprezentacije. Linije u dodatku mogu biti od pomoći. 
<br>
Podsetimo se da TfIdf reprezentacije uzimaju u obzir frekvencije pojavljivanja reči na nivou pojedinačnih rečenica (tf) otežane frekvencijama pojavljivanja na nivou svih rečenica (idf). Dužina ovakvih reprezentacija odgovara veličini vokabulara.   
</div>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit(clean_sentences)

In [ ]:
# vectorizer.get_feature_names()

In [ ]:
tfidf_vectors = vectorizer.transform(clean_sentences)

In [ ]:
N = len(sentences)
similarity_tfidf_matrix = np.zeros((N, N))
for i in range (0, N):
    for j in range (0, N):
        similarity_tfidf_matrix[i][j]= cosine_similarity(tfidf_vectors[i].reshape(1, -1), tfidf_vectors[j].reshape(1, -1))[0,0]